# Day 11

## Part 1

You continue following signs for "Hot Springs" and eventually come across an observatory. The Elf within turns out to be a researcher studying cosmic expansion using the giant telescope here.

He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight there once he's done with today's observation analysis.

Maybe you can help him with the analysis to speed things up?

The researcher has collected a bunch of data and compiled the data into a single giant image (your puzzle input). The image includes empty space (.) and galaxies (#). For example:

```
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
```
The researcher is trying to figure out the sum of the lengths of the shortest path between every pair of galaxies. However, there's a catch: the universe expanded in the time it took the light from those galaxies to reach the observatory.

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.

In the above example, three columns and two rows contain no galaxies:

```
   v  v  v
 ...#......
 .......#..
 #.........
>..........<
 ......#...
 .#........
 .........#
>..........<
 .......#..
 #...#.....
   ^  ^  ^
```
These rows and columns need to be twice as big; the result of cosmic expansion therefore looks like this:

```
....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
```
Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help to assign every galaxy a unique number:

```
....1........
.........2...
3............
.............
.............
........4....
.5...........
............6
.............
.............
.........7...
8....9.......
```
In these 9 galaxies, there are 36 pairs. Only count each pair once; order within the pair doesn't matter. For each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right exactly one . or # at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)

For example, here is one of the shortest paths between galaxies 5 and 9:

```
....1........
.........2...
3............
.............
.............
........4....
.5...........
.##.........6
..##.........
...##........
....##...7...
8....9.......
```
This path has length 9 because it takes a minimum of nine steps to get from galaxy 5 to galaxy 9 (the eight locations marked # plus the step onto galaxy 9 itself). Here are some other example shortest path lengths:

- Between galaxy 1 and galaxy 7: 15
- Between galaxy 3 and galaxy 6: 17
- Between galaxy 8 and galaxy 9: 5
In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is 374.

Expand the universe, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

In [5]:
import pandas as pd


def expand_universe(df: pd.DataFrame) -> pd.DataFrame:
    expanded_columns = []
    for i, col in enumerate(df.columns):
        # NOTE: we can't just `df.insert`, because if we have multiple columns missing galaxies,
        # we'll end up using an index value that's based on the original df, not the expanded one
        expanded_columns.append(df[col])
        if not df[col].sum():
            # no galaxies, add a new column to represent expansion
            expanded_columns.append(pd.Series(df[col], name=f"{col}e"))

    expanded_df = pd.concat(expanded_columns, axis=1)
    return expanded_df


def generate_universe(galaxy_data: str):
    universe_df = pd.DataFrame(
        [list(row) for row in galaxy_data.replace(".", "0").replace("#", "1").splitlines()]
    ).astype(int)

    # expand universal columns without galaxies
    universe_df = expand_universe(universe_df)
    # flip the universe to expand rows without galaxies
    universe_df = universe_df.transpose()
    universe_df = expand_universe(universe_df)
    # flip back
    universe_df = universe_df.transpose()

    return universe_df


def find_galaxies(universe_df: pd.DataFrame) -> list[tuple[int, int]]:
    galaxies = []
    for i, col in enumerate(universe_df.columns):
        for j, row in enumerate(universe_df[col]):
            if row:
                galaxies.append((i, j))
    return galaxies


def get_shortest_distance_between_galaxies(galaxy_locations: list[tuple[int, int]]) -> int:
    shortest_distances = {}
    for i, (x1, y1) in enumerate(galaxy_locations):
        for j, (x2, y2) in enumerate(galaxy_locations):
            if i == j:
                continue
            vertical_distance = abs(y1 - y2)
            horizontal_distance = abs(x1 - x2)
            total_distance = vertical_distance + horizontal_distance
            if (j, i) in shortest_distances:
                # we've already calculated this distance
                continue
            shortest_distances[(i, j)] = total_distance
    return shortest_distances


test_input = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

test_universe = generate_universe(test_input)
display(test_universe)
test_galaxy_locations = find_galaxies(test_universe)
test_shortest_galaxy_distances = get_shortest_distance_between_galaxies(test_galaxy_locations)
test_shortest_galaxy_distances, sum(test_shortest_galaxy_distances.values())

,0,1,2,2e,3,4,5,5e,6,7,8,8e,9
0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
3e,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0
7e,0,0,0,0,0,0,0,0,0,0,0,0,0


({(0, 1): 9,
  (0, 2): 5,
  (0, 3): 6,
  (0, 4): 14,
  (0, 5): 11,
  (0, 6): 10,
  (0, 7): 17,
  (0, 8): 17,
  (1, 2): 6,
  (1, 3): 15,
  (1, 4): 5,
  (1, 5): 14,
  (1, 6): 19,
  (1, 7): 10,
  (1, 8): 16,
  (2, 3): 9,
  (2, 4): 9,
  (2, 5): 8,
  (2, 6): 13,
  (2, 7): 12,
  (2, 8): 12,
  (3, 4): 12,
  (3, 5): 9,
  (3, 6): 6,
  (3, 7): 15,
  (3, 8): 15,
  (4, 5): 9,
  (4, 6): 14,
  (4, 7): 5,
  (4, 8): 11,
  (5, 6): 5,
  (5, 7): 6,
  (5, 8): 6,
  (6, 7): 9,
  (6, 8): 9,
  (7, 8): 6},
 374)

In [7]:
inputs = open("../inputs/11.txt").read()
universe = generate_universe(inputs)
galaxy_locations = find_galaxies(universe)
shortest_galaxy_distances = get_shortest_distance_between_galaxies(galaxy_locations)
sum(shortest_galaxy_distances.values())

9445168

## Part 2

The galaxies are much older (and thus much farther apart) than the researcher initially estimated.

Now, instead of the expansion you did before, make each empty row or column one million times larger. That is, each empty row should be replaced with 1000000 empty rows, and each empty column should be replaced with 1000000 empty columns.

(In the example above, if each empty row or column were merely 10 times larger, the sum of the shortest paths between every pair of galaxies would be 1030. If each empty row or column were merely 100 times larger, the sum of the shortest paths between every pair of galaxies would be 8410. However, your universe will need to expand far beyond these values.)

Starting with the same initial image, expand the universe according to these new rules, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

In [8]:
from rich import print as rprint


def generate_universe2(galaxy_data: str):
    universe_df = pd.DataFrame(
        [list(row) for row in galaxy_data.replace(".", "0").replace("#", "1").splitlines()]
    ).astype(int)
    return universe_df


def find_galaxies2(universe_df: pd.DataFrame, expansion_scale: int) -> list[tuple[int, int]]:
    galaxies = []
    for i, col in enumerate(universe_df.columns):
        for j, row in enumerate(universe_df[col]):
            if row:
                # count how many rows/columns exist before this one that don't have galaxies
                galaxyless_columns = [
                    col for col in universe_df.columns[:i] if not universe_df[col].sum()
                ]
                num_galaxyless_columns = len(galaxyless_columns)
                galaxyless_rows = [
                    col
                    for col in universe_df.transpose().columns[:j]
                    if not universe_df.transpose()[col].sum()
                ]
                num_galaxyless_rows = len(galaxyless_rows)

                # adjust the position based on the expansion scale and number of galaxyless rows/columns
                adjusted_x = i + (num_galaxyless_columns * expansion_scale)
                adjust_y = j + (num_galaxyless_rows * expansion_scale)

                galaxies.append((adjusted_x, adjust_y))
    return galaxies


def highlight_empty(styler):
    # highlight columns
    styler.apply(
        lambda x: [
            "background-color: #99000055" if x.values.sum() == 0 else None for v in x.values
        ],
        axis=0,
    )
    # highlight rows
    styler.apply(
        lambda x: [
            "background-color: #99000055" if x.values.sum() == 0 else None for v in x.values
        ],
        axis=1,
    )
    # highlight galaxies
    styler.apply(
        lambda x: ["background-color: #99990055" if v == 1 else None for v in x.values],
        axis=1,
    )
    return styler


test_universe2 = generate_universe2(test_input)
display(test_universe2.style.pipe(highlight_empty))
test_galaxy_locations2 = find_galaxies2(test_universe2, 9)
test_shortest_galaxy_distances2 = get_shortest_distance_between_galaxies(test_galaxy_locations2)
rprint(test_shortest_galaxy_distances2)

test_total_distances2 = sum(test_shortest_galaxy_distances2.values())
print(test_total_distances2)
assert test_total_distances2 == 1030

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0
5,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0
9,1,0,0,0,1,0,0,0,0,0


{
    (0, 1): 25,
    (0, 2): 13,
    (0, 3): 14,
    (0, 4): 38,
    (0, 5): 35,
    (0, 6): 26,
    (0, 7): 49,
    (0, 8): 49,
    (1, 2): 14,
    (1, 3): 39,
    (1, 4): 13,
    (1, 5): 38,
    (1, 6): 51,
    (1, 7): 26,
    (1, 8): 48,
    (2, 3): 25,
    (2, 4): 25,
    (2, 5): 24,
    (2, 6): 37,
    (2, 7): 36,
    (2, 8): 36,
    (3, 4): 28,
    (3, 5): 25,
    (3, 6): 14,
    (3, 7): 39,
    (3, 8): 39,
    (4, 5): 25,
    (4, 6): 38,
    (4, 7): 13,
    (4, 8): 35,
    (5, 6): 13,
    (5, 7): 14,
    (5, 8): 14,
    (6, 7): 25,
    (6, 8): 25,
    (7, 8): 22
}

1030


In [9]:
test_universe2b = generate_universe2(test_input)
test_galaxy_locations2b = find_galaxies2(test_universe2, 99)
test_shortest_galaxy_distances2b = get_shortest_distance_between_galaxies(test_galaxy_locations2b)
test_total_distances2b = sum(test_shortest_galaxy_distances2b.values())
print(test_total_distances2b)
assert test_total_distances2b == 8410

8410


In [10]:
universe2 = generate_universe2(inputs)
galaxy_locations2 = find_galaxies2(universe2, 999_999)
shortest_galaxy_distances2 = get_shortest_distance_between_galaxies(galaxy_locations2)
sum(shortest_galaxy_distances2.values())

742305960572